In [ ]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

# Data For PINNs

In [ ]:
N_train = 20000
N_bound = 200

np.random.seed(2345)
X_train = np.random.uniform(0,1,(N_train,2))
X_bound_zero_one = np.concatenate((np.zeros((int(N_bound/4),1)),np.ones((int(N_bound/4),1))),0)
np.random.seed(3456)
X_bound1 = np.concatenate((np.random.uniform(0,1,(int(N_bound/2),1)),X_bound_zero_one),1)
np.random.seed(5678)
X_bound2 = np.concatenate((X_bound_zero_one,np.random.uniform(0,1,(int(N_bound/2),1))),1)

X_bound = np.concatenate((X_bound1,X_bound2),0)

In [ ]:
#x,t  #u,v
X_train = torch.from_numpy(X_train).float()
X_bound = torch.from_numpy(X_bound).float()
x = X_train[:,0:1]
t = X_train[:,1:2]

### Labeled Data

In [ ]:
np.random.seed(1234)
k = 5
gan_data_x_t = lhs(2,3)
gan_data_x_t = torch.from_numpy(gan_data_x_t).float()
gan_data_u = torch.sin(k*gan_data_x_t[:,0:1])

### The Image With  Labeled Data

In [ ]:
traindata = np.concatenate((np.linspace(0,1,200).reshape(-1,1),np.linspace(0,1,200).reshape(-1,1)),1)
x_ = traindata[:,0:1]
y_ = traindata[:,1:2]
xx,yy = np.meshgrid(x_,y_)
data_numpy = np.concatenate((xx.reshape(-1,1),yy.reshape(-1,1)),1)
data_tensor = torch.from_numpy(data_numpy).float()
aa = torch.sin(k*data_tensor[:,0:1])

fig, ax = plt.subplots()
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1-0.06, bottom=1-1/3, left=0.15, right=0.85, wspace=0)
ax = plt.subplot(gs0[:, :])
ax.plot(gan_data_x_t[:,0:1], gan_data_x_t[:,1:2],  'kx',alpha=1,markersize = 4)
h = ax.imshow(aa.reshape(200,200), interpolation='nearest', cmap='rainbow',  extent=[0, 1, -1, 1],
            origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.show()

# Test Data

In [ ]:

np.random.seed(123)
test_x = torch.from_numpy(lhs(2,2000)).float()

In [ ]:
test_u = np.sin(k*test_x[:,0:1].numpy())

# Method2
**$\text { GA - PINN }$**

<div class="alert alert-info">`loss_function：`

$\mathrm{L}_D=\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(\mathbf{x}_T^{(j)}, u_T^{(j)}\right)\right]\right)+D\left[\left(x_L^{(j)}, G\left[x_L^{(j)}\right]\right)\right] \\
\mathrm{L}_G=\mathrm{L}_T+\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(x_T^{(j)}, G\left[x_T^{(j)}\right]\right)\right]\right)\\
\mathrm{L}_{\text {PINN }}:=\mathrm{L}_f+\lambda_1 \mathrm{~L}_b \text { with } \mathrm{L}_b:=\sum_{i=1}^I \mathrm{~L}_{b_i}
$



</div>

In [ ]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)


PINNs2 = NN_H2(2, 100, 4, 1)
PINNs2.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs2.parameters()}], lr=1e-3)
#optimizerg = optim.Adam([{'params': PINNs3.parameters()}], lr=1e-4)
nIter2 = 3000

discriminator = get_discriminator(3, 100, 1, 1)
discriminator.apply(weights_init)

optimizer2 = optim.Adam([{'params': discriminator.parameters(), 'weight_decay': 0.001}], lr=5e-5)


x.requires_grad_()
t.requires_grad_()

###########GPU###########
discriminator.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
real_bound = torch.sin(k*X_bound[:,0:1])
real_bound = real_bound.cuda()
X_bound = X_bound.cuda()
x = x.cuda()
t = t.cuda()
###########GPU###########


loss_all_2 = []
test_loss_2 = []

loss1_value = 1
it = 0
while loss1_value > 1e-2 and it < nIter2:
    PINNs2.cuda()
 
    if it<700:
        
        ##########loss D###########
        pre_H = PINNs2(gan_data_x_t)
        d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
        d_real = discriminator(torch.cat((gan_data_x_t,gan_data_u),1))

        loss_d = torch.mean(1-d_real)+torch.mean(d_fake)

        optimizer2.zero_grad()
        loss_d .backward()
        optimizer2.step()      

        ###########loss G###########
        for param_group in optimizer1.param_groups:
            param_group["lr"]=1e-5
        pre_H = PINNs2(gan_data_x_t)
        d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
        loss_g = (torch.mean(1-d_fake)+torch.mean(torch.square(pre_H - gan_data_u)))

        optimizer1.zero_grad()
        loss_g.backward()
        optimizer1.step()

    ###########loss PINN#########
    for param_group in optimizer1.param_groups:
        param_group["lr"]=1e-3
        
    ##### loss_Bi  ######
    E_bound = PINNs2(X_bound)
    loss_bound = torch.mean(torch.square(E_bound-real_bound))
    

    ##### loss f  ######
    E_inside = PINNs2(torch.cat((x,t),1))
    E_x = autograd.grad(outputs=E_inside, inputs=x,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_xx = autograd.grad(outputs=E_x, inputs=x,
                              grad_outputs=torch.ones_like(E_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_t = autograd.grad(outputs=E_inside, inputs=t,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_tt = autograd.grad(outputs=E_t, inputs=t,
                              grad_outputs=torch.ones_like(E_t),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]       
    deata_E = E_xx+E_tt
    loss_f = torch.mean(torch.square(deata_E+k*k*E_inside))
     
        
    ##### loss PI #######
    loss = 20*loss_bound+loss_f
    
    loss_real = loss
    
    loss_all_2.append(loss_real.item())
    
    loss1_value = loss_real.item()
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs2.cpu()
    e1 = relative_l2(PINNs2(test_x).detach().numpy(),test_u)
    test_loss_2.append(e1)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss_real.item())
    it = it + 1        
loss1_value    

# Results

## Save data

In [ ]:
np.save('../experimental_data/method_2/test_loss_2',test_loss_2)
np.save('../experimental_data/method_2/loss_all_2',loss_all_2)
torch.save(PINNs2,'../saved_model/PINNs2')

## Epoch and NRMSE

In [ ]:
Epochs = len(test_loss_2)
NRMSE = relative_l2(PINNs2(test_x).detach().numpy(),test_u)

print('Epochs:',Epochs,'NRMSE:',NRMSE)